# Evaluation Metrics

In [2]:
import pandas as pd
import numpy as np
import copy
import os
from scipy.stats import friedmanchisquare
from scipy.stats import chi2

# change to your local path to project below
project_path = '/home/jorge/Insync/jorgitoje@gmail.com/OneDrive/Documentos/JORGE/EDUCATION/MASTER_DATASCIENCE/Semester1/AdvancedStatistics/GroupProject/KDS_Statistics_GroupProject'
# TO DO:
# - Include CART method when already having data correct
# - Add tie correction to W
# -  

In [34]:
featureranks_basepath = os.path.join(project_path, 'data')
all_ranks_all_methods_long = pd.DataFrame()

# Put all ranks into one csv
for file in os.listdir(featureranks_basepath):
    if file.endswith('.csv') and file != 'featureranks_ALLMETHODS.csv':
        df = pd.read_csv(os.path.join(featureranks_basepath, file), sep=";")
        all_ranks_all_methods_long = pd.concat([all_ranks_all_methods_long, df], axis=0)

# Save all ranks together
all_ranks_all_methods_long.to_csv(os.path.join(project_path, 'data', 'featureranks_ALLMETHODS.csv'), sep=";", index=False)
all_ranks_all_methods_long = all_ranks_all_methods_long.query('method != "CART" and method != "Lasso_regression"') # temporarily take out these methods


dataset  feature_name      feature_rank                \
method                             Linear_regression Random_forest   
0       Airbnb_price  accommodates               5.0           7.0   
1       Airbnb_price     amenities              16.5           5.0   
2       Airbnb_price     bathrooms               2.0           3.0   
3       Airbnb_price      bed_type              13.0          17.0   
4       Airbnb_price      bedrooms               3.0           8.0   

                                     
method Ridge_regression elastic_net  
0                   5.0         4.0  
1                  16.5        13.0  
2                   2.0         3.0  
3                  13.0        14.0  
4                   3.0         2.0

In [33]:
all_ranks_all_methods_long.query('method != "CART" and method != "Lasso_regression"').groupby(['method', 'dataset'])['feature_name'].count()

method             dataset             
Linear_regression  Airbnb_price            18
                   Bike_sharing            12
                   USA_houseprices_2014    13
                   austin_housing          27
                   cars                    34
Random_forest      Airbnb_price            18
                   Bike_sharing            12
                   USA_houseprices_2014    13
                   austin_housing          27
                   cars                    34
Ridge_regression   Airbnb_price            18
                   Bike_sharing            12
                   USA_houseprices_2014    13
                   austin_housing          27
                   cars                    34
elastic_net        Airbnb_price            18
                   Bike_sharing            12
                   USA_houseprices_2014    13
                   austin_housing          27
                   cars                    34
Name: feature_name, dtype: int64

## Kendall's W

### Using friedmanchisquare
(DOES NOT seem to work and requires manually putting all columns)

In [18]:

# spRes = friedmanchisquare(data_kendall['PCA'].to_numpy(), data_kendall['RandomForest'].to_numpy(), data_kendall['linear_regression'].to_numpy())
# spRes = friedmanchisquare(*[data_kendall[column] for column in data_kendall.columns])
# friedmanchisquare()
# selData()
# n = data_kendall.shape[0]
# k = data_kendall.shape[1]
# Q = spRes[0]
# print(f"n: {n},k: {k}, Q: {Q}")

In [19]:
# W = Q / (n*(k-1))
# print(f"W: {W}")

### Using direct formula

In [46]:
def calculate_kendallW(data_kendall, tie_correction=False):
    '''_summary_

    Args:
        data_kendall (pd.dataframe): Datafrem containing as columns the feature
        selection methods and as values ranks to each feature in the same order.

    Returns:
        W: Kendall's W coefficient
    '''
    n = data_kendall.shape[0] # number of features/objects
    k = data_kendall.shape[1] # numbe of raters/methods

    # Sum of each item ranks
    sums = data_kendall.sum(axis=1, numeric_only=True).to_numpy()
    # Mean of ranking sums
    Rbar = sums.mean()
    Rbar
    # Sum of squared deviations from the mean
    S = np.sum([(np.array(sums)[x] - Rbar) ** 2 for x in range(n)])
    if not tie_correction:
        W = (12 * S) / (k ** 2 * (n ** 3 - n))
    else:
        # TO DO: implement tie correction
        T=3 
        numerator = 12*np.sum([np.array(sums)[x] for x in range(n)]) - 3*(k**2)*n*(n+1)**2
        print(numerator)
        denominator = ((k**2)*n*(n**2 - 1) - k*T)
        W = numerator /  denominator
        return None

    return W

## Top–k overlap

In [49]:
def calculate_topkoverlap(datatopk, k=5):

    data_all = datatopk.copy(deep=True)

    n = len(data_all)

    assert len(data_all) >= k, f"Maximum value for k is {n}, which is total number of features"

    methods = data_all.columns.to_list()
    data_all['feature_id'] = data_all.index

    all_toplists = []

    for method in methods:
        data_temp = data_all.filter(items=['feature_id', method]) \
            .sort_values(method).head(k)['feature_id'].to_list()
        all_toplists.append(data_temp)
    union_lists= np.unique(all_toplists)
    intersection_lists = set.intersection(*map(set, all_toplists))

    topk_overlap = len(intersection_lists)/len(union_lists)
        
    return topk_overlap


## Hypothesis testing for all datasets

In [61]:
for dataset_name in all_ranks_all_methods_long['dataset'].unique():

    # Transform to long for better manipulation
    data_ranks_wide = all_ranks_all_methods_long.query('dataset == @dataset_name').pivot( index=['dataset', 'feature_name'], columns=['method'], 
        values=['feature_rank']).reset_index()
    data_ranks_wide.head()

    data_kendall = data_ranks_wide['feature_rank'].astype(int)


    W = calculate_kendallW(data_kendall, tie_correction=False)
    top5_overlap = calculate_topkoverlap(data_kendall, k=5)
    top10_overlap = calculate_topkoverlap(data_kendall, k=10)

    n = data_kendall.shape[0]
    k = data_kendall.shape[1]

    alpha = 0.05
    degrees_of_freedom = n - 1
    chisquared_alpha = chi2.ppf(q = alpha, df = degrees_of_freedom)
    chisquared_statistic = k *(n-1) * W

    print(f"RESULTS FOR DATASET '{dataset_name}'")
    print(f"There are {n} features and {k} raters")
    print(f"Kendall's W: {W}")
    print(f"Top-5 overlap: {top5_overlap}")
    print(f"Top-10 overlap: {top10_overlap}")
    print(f"Statistic value(alpha={alpha}): {chisquared_statistic}")
    print(f"Chi2 (alpha={alpha}, {degrees_of_freedom} dof): {chisquared_alpha}") # Tabled value

    if chisquared_statistic >= chisquared_alpha:
        print("We reject null hypothesis and conclude that H1: W ≠ 0.")
    else:
        print("We cannot reject null hypothesis H0: W = 0")
    
    print('-------------------------------------------------------------------')

RESULTS FOR DATASET 'USA_houseprices_2014'
There are 13 features and 4 raters
Kendall's W: 0.5675718512256974
Top-5 overlap: 0.1111111111111111
Top-10 overlap: 0.5384615384615384
Statistic value(alpha=0.05): 27.243448858833474
Chi2 (alpha=0.05, 12 dof): 5.226029488392639
We reject null hypothesis and conclude that H1: W ≠ 0.
-------------------------------------------------------------------
RESULTS FOR DATASET 'Bike_sharing'
There are 12 features and 4 raters
Kendall's W: 0.7801209207459208
Top-5 overlap: 0.6666666666666666
Top-10 overlap: 0.6666666666666666
Statistic value(alpha=0.05): 34.32532051282051
Chi2 (alpha=0.05, 11 dof): 4.574813079322224
We reject null hypothesis and conclude that H1: W ≠ 0.
-------------------------------------------------------------------
RESULTS FOR DATASET 'Airbnb_price'
There are 18 features and 4 raters
Kendall's W: 0.5490769407178075
Top-5 overlap: 0.2222222222222222
Top-10 overlap: 0.3333333333333333
Statistic value(alpha=0.05): 37.33723196881091
C